# Проект для «Викишоп» 

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import numpy as np
import pandas as pd
import torch
from tqdm import notebook
import transformers as ppb 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
import time
import re
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')

Прочитали данные, загрузили предобученную модель distilbert, для экономии времени ограничим выборку до 16000 строк

In [3]:
wnl = WordNetLemmatizer()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(wnl.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

df['lemm_text'] = df['text'].apply(lemmatize_text)
text = df['lemm_text'] 
df = df.drop(['text'], axis=1)

In [4]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
target = df['toxic']
features = df.drop(['toxic'], axis=1)


0    143346
1     16225
Name: toxic, dtype: int64

In [6]:
train_features, test_features, train_labels, test_labels = train_test_split(features,
                                                                            target, test_size=0.2, random_state=123456)

In [7]:
train_features = count_tf_idf.fit_transform(train_features['lemm_text'].values.astype('U'))
test_features = count_tf_idf.transform(test_features['lemm_text'].values.astype('U'))

In [8]:
train_start = time.time()
model = LGBMClassifier() 

hyperparams = [{'learning_rate':[0.1,  0.5, 0.9],
                'max_depth':[x for x in range(1, 10,2)],
                'random_state':[123456]}]


clf = GridSearchCV(model, hyperparams, scoring='f1')
clf.fit(train_features, train_labels)
print("Лучшие гиперпараметры модели:")

print(clf.best_params_)
train_end = time.time()
print('Время подбора гиперпараметров:{}'.format(train_end - train_start))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Лучшие гиперпараметры модели:
{'learning_rate': 0.5, 'max_depth': 9, 'random_state': 123456}
Время подбора гиперпараметров:2511.76433134079


In [12]:
train_start = time.time()
model = LogisticRegression() 

hyperparams = [{'solver':['lbfgs', 'liblinear'],
               'max_iter' :  [10000],              
               }]


clf = GridSearchCV(model, hyperparams, scoring='f1')
clf.fit(train_features, train_labels)
print("Лучшие гиперпараметры модели:")

print(clf.best_params_)
train_end = time.time()
print('Время подбора гиперпараметров:{}'.format(train_end - train_start))

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Лучшие гиперпараметры модели:
{'max_iter': 10000, 'solver': 'lbfgs'}
Время подбора гиперпараметров:236.48582577705383


Подобрали оптимальные гиперпараметры моделей.

In [13]:

models = [LGBMClassifier( learning_rate = 0.5, max_depth = 9, random_state= 123456), 
          LogisticRegression(solver = 'lbfgs', max_iter=10000)]


results = []

for model in models: 
    train_start = time.time()  
    model.fit(train_features, train_labels)
    train_end = time.time()
    predict_start = time.time()
    pred = model.predict(test_features)
    predict_end = time.time()
    f1 = f1_score(test_labels, pred)
    results.append({'model_name': model.__class__.__name__, 
                    'training time ': train_end - train_start,
                    'prediction time ': predict_end - predict_start,
                    'f1': f1                              
                              })  
              
res = pd.DataFrame(results)
res

,model_name,training time,prediction time,f1
0,LGBMClassifier,137.943609,1.659317,0.759070
1,LogisticRegression,51.366205,0.007830,0.732064


При схожих времени обучения и предказания, значение f1 больше у LGBMClassifier. Советуем использовать ее для предсказания.